In [9]:
#Imports
import os
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime
import pickle
import tqdm
import line_profiler
from operator import itemgetter
%load_ext heat

In [11]:
def track_id_gen(dist, track):
    return track[0:4]+"-"+str(int(dist))

In [12]:
#pickle Load results
resultspicklefile = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Data Processed\results-df-w-dates.npy', 'rb')
resultsdf = pickle.load(resultspicklefile)
resultsdf = pd.DataFrame(resultsdf)
resultsdf.head()
split_distances = pd.read_csv("split_distance.csv")
resultsdf["dist"] = resultsdf["dist"].astype(str).str[:-1].astype(float)

resultsdf.run_time = resultsdf.run_time.astype(float)
resultsdf["track_id"] = resultsdf.apply(lambda s: track_id_gen(s["dist"],s["track"]), axis=1)
resultsdf_merged = pd.merge(resultsdf, split_distances, on="track_id")

In [13]:
resultsdf_merged["date"] = pd.to_datetime(resultsdf_merged["date"], format="%d %b %y")
resultsdf_merged["speed"] = pd.to_numeric(resultsdf_merged["run_time"])/pd.to_numeric(resultsdf_merged["dist_x"])
resultsdf_merged["split_speed"] = pd.to_numeric(resultsdf_merged["split_times"])/pd.to_numeric(resultsdf_merged["split_dist_estim"])
resultsdf_merged["box"] = pd.to_numeric(resultsdf_merged["box"])
resultsdf_merged

,race_id,date,track_x,dist_x,grade,place,dog_id,box,split_times,split_margins,...,Unnamed: 0,track_y,dist_y,mean_time,mean_split,split_dist_estim,split_dist,id,speed,split_speed
0,208756700,2017-04-01,Ballarat,450.0,Maiden Heat,1.0,1296100098,2,6.73,0.00,...,2,Ballarat,450,26.038262,6.791825,117.378074,0,Ball-450,0.058156,0.057336
1,208756700,2017-04-01,Ballarat,450.0,Maiden Heat,2.0,1177320005,3,6.87,0.14,...,2,Ballarat,450,26.038262,6.791825,117.378074,0,Ball-450,0.058178,0.058529
2,208756700,2017-04-01,Ballarat,450.0,Maiden Heat,3.0,1549980026,5,6.99,0.26,...,2,Ballarat,450,26.038262,6.791825,117.378074,0,Ball-450,0.058356,0.059551
3,208756700,2017-04-01,Ballarat,450.0,Maiden Heat,4.0,1474670055,7,6.88,0.15,...,2,Ballarat,450,26.038262,6.791825,117.378074,0,Ball-450,0.058578,0.058614
4,208756700,2017-04-01,Ballarat,450.0,Maiden Heat,5.0,913850094,8,6.91,0.18,...,2,Ballarat,450,26.038262,6.791825,117.378074,0,Ball-450,0.058711,0.058870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450056,224547720,2017-05-30,Warragul,680.0,Special Event,3.0,954250096,2,18.15,0.00,...,48,Warragul,680,39.893460,18.804604,320.532003,0,Warr-680,0.058059,0.056625
450057,224547720,2017-05-30,Warragul,680.0,Special Event,4.0,1428630000,3,19.10,0.95,...,48,Warragul,680,39.893460,18.804604,320.532003,0,Warr-680,0.058191,0.059588
450058,224547720,2017-05-30,Warragul,680.0,Special Event,5.0,-743571,4,18.83,0.68,...,48,Warragul,680,39.893460,18.804604,320.532003,0,Warr-680,0.058221,0.058746
450059,224547720,2017-05-30,Warragul,680.0,Special Event,6.0,-748349,6,19.03,0.88,...,48,Warragul,680,39.893460,18.804604,320.532003,0,Warr-680,0.058676,0.059370


In [7]:
def input_layer_gen(form):
    race_forms = form.groupby(["race_id"])
    for i,j in race_forms:
        print(i,j)
        break

In [17]:
#%%heat
#pickle Load results
resultspicklefile = open( r'C:\Users\Nick\Documents\GitHub\grvmodel\Data Processed\results-df-w-dates.npy', 'rb')
resultsdf = pickle.load(resultspicklefile)
resultsdf = pd.DataFrame(resultsdf)
resultsdf.head()
split_distances = pd.read_csv("split_distance.csv")
resultsdf["dist"] = resultsdf["dist"].astype(str).str[:-1].astype(float)

resultsdf.run_time = resultsdf.run_time.astype(float)
resultsdf["track_id"] = resultsdf.apply(lambda s: track_id_gen(s["dist"],s["track"]), axis=1)
resultsdf_merged = pd.merge(resultsdf, split_distances, on="track_id")
form = resultsdf_merged
race_forms = form.groupby(["race_id"])
print(race_forms)
race_input_layer = []
race_classes = []

resultsdf_merged["date"] = pd.to_datetime(resultsdf_merged["date"], format="%d %b %y")
resultsdf_merged["speed"] = pd.to_numeric(resultsdf_merged["run_time"])/pd.to_numeric(resultsdf_merged["dist_x"])
resultsdf_merged["split_speed"] = pd.to_numeric(resultsdf_merged["split_times"])/pd.to_numeric(resultsdf_merged["split_dist_estim"])
resultsdf_merged["box"] = pd.to_numeric(resultsdf_merged["box"])
resultsdf_merged

for i,j in tqdm.tqdm(race_forms):
    try:
        race_date = pd.to_datetime(j["date"].iloc[0])
        #print(f"{race_date=}")
        dogs = j.dog_id
        #print(f"race id {i=}")
        race_dist = j.dist_x.iloc[1]
        dog_info = []
        i = 0
        for dog in dogs:
            #print(f"{dog=}")
            prev_form = form[(form.dog_id == dog)&(form["date"]< race_date)]
            dog_speed_avg = prev_form.speed.mean()
            dog_split_avg = prev_form.split_speed.mean()
            dog_race_count = prev_form.shape[0]
            dog_box = j.box.iloc[i]
            dog_stats = (dog, [dog_box,dog_speed_avg,dog_split_avg,dog_race_count], j.place.iloc[i])
            dog_info.append(dog_stats)
            #print(f"{dog_stats=}")
            i = i+1
        dog_info.sort(key=itemgetter(0))
        input_layer = [item for sublist in [x[1] for x in dog_info] for item in sublist]
        classes = [x[2] for x in dog_info]
        race_input_layer.append(input_layer)
        race_classes.append(classes)
    except Exception as e:
        print(f"{e=}")

 88%|████████▊ | 49145/55828 [2:42:48<17:39,  6.31it/s]  

e=IndexError('single positional indexer is out-of-bounds')


100%|██████████| 55828/55828 [3:09:14<00:00,  4.92it/s]  


race_date=Timestamp('2018-06-22 00:00:00')
race id i='12539914'
dog='115367782'
dog_stats=('115367782', ['8', 0.05848506493506494, 0.058515294245794845, 11], 1.0)
dog='128006011'
dog_stats=('128006011', ['5', 0.05816878881987578, 0.05788880463785036, 4], 2.0)
dog='167542442'
dog_stats=('167542442', ['4', 0.05878095238095238, 0.05877252045258206, 1], 3.0)
dog='145856635'
dog_stats=('145856635', ['2', 0.05965141007859454, 0.059659061022691785, 6], 4.0)
dog='159734832'
dog_stats=('159734832', ['7', 0.05808414021919448, 0.0589491689599934, 10], 5.0)
dog='2131520028'
dog_stats=('2131520028', ['3', 0.059227187313647844, 0.06019035872786623, 6], 6.0)
dog='1199430051'
dog_stats=('1199430051', ['6', 0.061256388082696595, 0.06081872302453238, 9], 7.0)
dog='109032142'
dog_stats=('109032142', ['1', 0.059672102206736355, 0.05890175351276232, 5], 8.0)


In [18]:
with open("input_list.npy", "wb") as fp:   #Pickling
    
    pickle.dump(race_input_layer, fp)

with open("classes_list.npy", "wb") as fp:   #Pickling
    
    pickle.dump(race_classes, fp)

In [27]:
d = race_date
d

Timestamp('2016-07-01 00:00:00')